# Inferring the goals of autonomous agents in Gen.jl

what is Gen.jl (2 sentences)

In [1]:
using Gen
srand(1);

## 1. Modeling an autonomous agent

Explain ( 2 sentence). Explain that a model is a probabilisttic program, and that @program is used to define a probabilistic program in Gen.jl

In [2]:
include("scene.jl")
include("path_planner.jl");

In [3]:
@program agent_model() begin
    
    # assumed scene
    scene = Scene(0, 100, 0, 100) # the scene spans the square [0, 100] x [0, 100]
    add!(scene, Tree(Point(30, 20))) # place a tree at x=30, y=20
    add!(scene, Tree(Point(83, 80)))
    add!(scene, Tree(Point(80, 40)))
    wall_height = 30.
    add!(scene, Wall(Point(20., 40.), 1, 40., 2., wall_height))
    add!(scene, Wall(Point(60., 40.), 2, 40., 2., wall_height))
    add!(scene, Wall(Point(60.-15., 80.), 1, 15. + 2., 2., wall_height))
    add!(scene, Wall(Point(20., 80.), 1, 15., 2., wall_height))
    add!(scene, Wall(Point(20., 40.), 2, 40., 2., wall_height))    
    
    # time points at which we observe the agent's location
    observation_times = collect(linspace(0.0, 200.0, 20)) ~ "times"
    
    # assumed speed of the agent
    speed = 1.0
    
    # the starting location of the agent is a random point in the scene
    start = Point(uniform(0, 100), uniform(0, 100)) ~ "start"
    
    # the destination of the agent is a random point in the scene
    destination = Point(uniform(0, 100), uniform(0, 100)) ~ "destination"
    
    # the path of the agent from its start location to its destination
    # uses a simple 2D holonomic path planner based on RRT (path_planner.jl)
    (tree, rough_path, final_path) = plan_path(start, destination, scene)
    
    if isnull(final_path)
        
        # the agent could not find a path to its destination
        # assume it stays at the start location indefinitely
        locations = [start for _ in observation_times]
    else
        
        # the agent found a path to its destination
        # assume it moves from the start to the destinatoin along the path at constnat speed
        # sample its location along this path for each time in observation times
        locations = walk_path(get(final_path), speed, observation_times)
    end
    
    # assume that the observed locations are noisy measurements of the true locations
    # assume the noise is normally distributed with standard deviation 'noise'
    noise = 1.0
    for (i, t) in enumerate(observation_times)
        measured_x = normal(locations[i].x, noise) ~ "x$i"
        measured_y = normal(locations[i].y, noise) ~ "y$i"
    end
    
    # record other program state for rendering
    scene ~ "scene"
end;

We can run the model to generate probable scenarios:. We run a model using `@generate`, which executes a program and populates a trace with the values of the named expressions in the program.

EXPLAIN THAT ANY PROGRAM STATE CAN BE TRACED INCLUDING RANDOM CHOICES AND NON-RANDOM CHOICES

In [4]:
trace = Trace()
@generate(trace, agent_model())
println("start: ", value(trace, "start"))
println("destination: ", value(trace, "destination"))
println("x1 through x4: ", map((i) -> value(trace, "x$i"), 1:4))
println("y1 through y4: ", map((i) -> value(trace, "y$i"), 1:4))

start: Point(23.603334566204694,34.651701419196044)
destination: Point(31.27069683360675,0.790928339056074)
x1 through x4: [22.9397,23.702,26.2668,29.5669]
y1 through y4: [34.8259,23.4063,14.2533,4.04189]


If we run it again, we get a different result:

In [5]:
trace = Trace()
@generate(trace, agent_model())
println("start: ", value(trace, "start"))
println("destination: ", value(trace, "destination"))
println("x1 through x4: ", map((i) -> value(trace, "x$i"), 1:4))
println("y1 through y4: ", map((i) -> value(trace, "y$i"), 1:4))

start: Point(6.142672559357476,11.794967166048353)
destination: Point(4.3125750752229575,91.04863565786087)
x1 through x4: [5.58569,5.13517,5.16809,6.02647]
y1 through y4: [12.039,21.4391,33.0913,44.1358]


We can view the whole trace. For now, don't worry about constraints, interventions, or proposals. Note that the `recorded` section lists all the values that were recorde. 

In [6]:
print(trace)

-- Constraints --
-- Interventions --
-- Proposals --
-- Recorded --
times => [0.0,10.5263,21.0526,31.5789,42.1053,52.6316,63.1579,73.6842,84.2105,94.7368,105.263,115.789,126.316,136.842,147.368,157.895,168.421,178.947,189.474,200.0]
start => Point(6.142672559357476,11.794967166048353)
destination => Point(4.3125750752229575,91.04863565786087)
x1 => 5.585688455826619
y1 => 12.038992356552045
x2 => 5.1351724784263615
y2 => 21.43906821965481
x3 => 5.168094059970332
y3 => 33.09126239631681
x4 => 6.026469498642951
y4 => 44.135812822414984
x5 => 6.073903301658318
y5 => 54.88665362694848
x6 => 3.6052589248024507
y6 => 64.23047781256096
x7 => 3.8543651489948436
y7 => 74.83851321808031
x8 => 4.492965811003994
y8 => 85.50181517895942
x9 => 5.005810818840624
y9 => 90.62758163710411
x10 => 3.8011450231810953
y10 => 91.65321296417589
x11 => 7.0808138134763885
y11 => 91.16896924195844
x12 => 3.6120677543535464
y12 => 91.54973411423751
x13 => 3.419312563112474
y13 => 88.50124859453108
x14 => 3.77563

# 2. Visualizing the probabilistic behavior of a model using a trace rendering

Printing out the values of variables is not a very good way to understand the probabilistic behavior of a program. Instead, we use a **trace rendering** to produce a visual representation of the trace. The trace rendering encodes the trace into a representation that the human visual system can quickly interpret. In Gen.jl a trace renderer is simply object that has a method `render(trace::Trace)`. In Jupyter notebooks, we render traces using JavaScript code with a generic `JupyterInlineRenderer`. 

FIX ME


In [7]:
javascript"""

var Gen = require("nbextensions/gen_notebook_extension/main");
var d3 = require("nbextensions/d3/d3.min");

function add_svg(parent, trace) {
    return parent.append("svg")
        .attr("viewBox", "0 0 100 100") // TODO viewBox should depend on scene limits
        .attr("position", "absolute")
        .style("height", "100%");
}

function add_svg_if_not_exists(parent, trace) {
    var svg = parent.selectAll("svg").data([""]);
    return svg.enter().append("svg")
        .attr("viewBox", "0 0 100 100") // TODO viewBox should depend on scene limits
        .attr("position", "absolute")
        .style("height", "100%")
        .merge(svg);
}

function add_bounding_box(svg) {
    svg.selectAll("rect").data([""]).enter().append("rect")
        .attr("width", "100%")
        .attr("height", "100%")
        .attr("stroke", "#000")
        .attr("fill", "#FFF")
        .attr("fill-opacity", 0.0);
}

function add_scene(svg, trace) {
    var scene = Gen.find_choice(trace, "scene");
    var trace_trees = scene.value.obstacles.filter(function(element) { return element.name == "Tree";});
    var trees = svg.selectAll(".tree").data(trace_trees);
    trees.exit().remove();
    trees.enter().append("rect")
        .style("fill", "green")
        .classed("tree", true)
      .merge(trees)
        .attr("x", function(d) { return d.center.x - d.size/2.0; })
        .attr("y", function(d) { return d.center.y - d.size/2.0; })
        .attr("width", function(d) { return d.size; })
        .attr("height", function(d) { return d.size; });
    var trace_walls = scene.value.obstacles.filter(function(element) { return element.name == "Wall";});
    var walls = svg.selectAll(".wall").data(trace_walls);
    walls.exit().remove();
    walls.enter().append("rect")
        .style("fill", "gray")
        .classed("wall", true)
      .merge(walls)
        .attr("x", function(d) { return d.start.x; })
        .attr("y", function(d) { return d.start.y; })
        .attr("width", function(d) { return d.orientation == 1 ? d.length : d.thickness; })
        .attr("height", function(d) { return d.orientation == 2 ? d.length : d.thickness; });
}

function add_start(svg, trace) {
    // the starting position of the agent
    var radius = 2;
    var trace_start = Gen.find_choice(trace, "start");
    var start = svg.selectAll(".start").data(trace_start ? [trace_start] : []);
    start.exit().remove();
    start.enter().append("circle")
        .attr("r", radius)
        .style("fill", "blue")
        .classed("start", true)
      .merge(start)
        .attr("cx", function(d) { return d.value.x; })
        .attr("cy", function(d) { return d.value.y; });
}

function add_destination(svg, trace) {
    // the destination position of the agent
    var radius = 2;
    var trace_dest = Gen.find_choice(trace, "destination");
    var dest = svg.selectAll(".destination").data(trace_dest ? [trace_dest] : []);
    dest.exit().remove();
    dest.enter().append("circle")
        .attr("r", radius)
        .style("fill", "red")
        .classed("destination", true)
      .merge(dest)
        .attr("cx", function(d) { return d.value.x; })
        .attr("cy", function(d) { return d.value.y; });
}

function add_path(svg, trace, update) {
    // the path points of the agent
    // TODO would be great if they could be observed together..
    var radius = 2;
    var times = Gen.find_choice(trace, "times").value;
    var path_point_data = [];
    for (var i=1; i<=times.length; i++) {
        path_point_data.push({x: Gen.find_choice(trace, "x" + i),
                              y: Gen.find_choice(trace, "y" + i)});
    }
    var path_segment_data = [];
    for (var i=0; i<times.length-1; i++) {
        path_segment_data.push({prev: {x: path_point_data[i].x,   y: path_point_data[i].y},
                                next: {x: path_point_data[i+1].x, y: path_point_data[i+1].y}});
    }

    // if update, join on a key that includes which trace 
    // we should be able to store the age of an element in the DOM?
    // and increment the age each time?
    var path_segments = svg.selectAll(".path_segments").data(path_segment_data);
    path_segments.exit().remove();
    path_segments.enter().append("line")
        .style("stroke", "#000")
        .classed("path_segments", true)
      .merge(path_segments)
        .attr("x1", function(d) { return d.prev.x.value; })
        .attr("y1", function(d) { return d.prev.y.value; })
        .attr("x2", function(d) { return d.next.x.value; })
        .attr("y2", function(d) { return d.next.y.value; });
    
    var radius = 2;
    var path_points = svg.selectAll(".path").data(path_point_data);
    path_points.exit().remove();
    path_points.enter().append("circle")
        .attr("r", radius)
        .style("fill", "orange")
        .style("fill-opacity", 0.5)
        .classed("path", true)
        .on("mouseover", handleMouseOver)
        .on("mouseout", handleMouseOut)
      .merge(path_points)
        .attr("cx", function(d) { return d.x.value; })
        .attr("cy", function(d) { return d.y.value; });
    svg.selectAll(".path")
        .classed("interventions", function(d) { return d.x.where == Gen.interventions || d.y.where == Gen.interventions; })
        .classed("constraints", function(d) { return d.x.where == Gen.constraints || d.y.where == Gen.constraints; });
        
    function handleMouseOver(d, i) {
        var x = d.x.value;
        var y = d.y.value;
         // Specify where to put label of text
        svg.append("text").attr("id", "t" + "-" + i).attr("x", x).attr("y", y)
            .attr("font-size", "6px")
            .attr("pointer-events", "none")
            .text([x.toFixed(1), y.toFixed(1)]);
    };
                              
    function handleMouseOut(d, i) {
        var x = d.x.value;
        var y = d.y.value;
         // Select text by id and then remove
        d3.select("#t" + "-" + i).remove();
    };
}

function apply_styles(svg) {
    // whether the start and destination are intervened or constrained
    svg.selectAll(".destination, .start")
        .classed("interventions", function(d) { return d.where == Gen.interventions; })
        .classed("constraints", function(d) { return d.where == Gen.constraints; });

    // apply styles to indicate intervened or constrained
    svg.selectAll(".interventions")
        .style("stroke", "#000")
        .style("stroke-width", 1);
    svg.selectAll(".constraints")
        .style("stroke", "#000")
        .style("stroke-width", 1)
        .style("stroke-dasharray", "1, 1");
}

function add_log_score(svg, trace) {
    var score = svg.selectAll(".score").data([""]);
    score.enter().append("text")
        .classed("score", true)
        .attr("x", 50).attr("y", 95).attr("text-anchor", "middle")
        .attr("font-size", "10px")
      .merge(score)
        .text(trace.log_weight.toFixed(2));
}

Gen.register_jupyter_renderer("agent_model_renderer", function(id, trace, conf) {
    var root = d3.select("#" + id);
    var svg;
    switch (conf.mode) {
        case "overlay":
            root = add_svg_if_not_exists(root, trace);
            svg = add_svg(root, trace);
            break;
        case "overwrite":
            svg = add_svg_if_not_exists(root, trace);
            break;
        case "tile":
            svg = add_svg(root, trace);
            break;
        default:
            break;
    }
    add_bounding_box(svg);
    if (conf.show_path) {
        add_path(svg, trace, false); // add a version that keeps the old one around (and dims it?)
    }
    add_start(svg, trace);
    add_destination(svg, trace);
    apply_styles(svg);
    add_scene(svg, trace);
    if (conf.show_score) {
        add_log_score(svg, trace);
    }
});
"""

In [8]:
figure = Figure(width=200, height=200, trace_width=100, trace_height=100, margin_top=20)
here(figure)

HTML{String}("    <div id=\"id_8cc2dc6ac6144e56b8bc403ff57ef106\"></div>\n    <script>\n        var d3 = require(\"nbextensions/d3/d3.min\");\n    \n        var svg = d3.select(\"#id_8cc2dc6ac6144e56b8bc403ff57ef106\").append(\"svg\")\n            .attr(\"width\", 200)\n            .attr(\"height\", 200);\n    \n        // TODO delete me\n        svg.append(\"rect\")\n            .attr(\"x\", 0)\n            .attr(\"y\", 0)\n            .attr(\"width\", \"100%\")\n            .attr(\"height\", \"100%\")\n            .style(\"stroke\", \"white\")\n            .style(\"fill\", \"white\");\n    \n        var tile_width = 200 / 1;\n        var tile_height = 200 / 1;\n        var i = 1;\n        for (var row=0; row<1; row++) {\n            for (var col=0; col<1; col++) {\n                var sub_svg = svg.append(\"svg\")\n                    .classed(\"subfigure\", true)\n\t\t\t        .attr(\"id\", \"id_8cc2dc6ac6144e56b8bc403ff57ef106_parent\" + i)\n                    .attr(\"x\", col * tile_width)\n                    .attr(\"y\", row * tile_height)\n                    .attr(\"width\", tile_width)\n                    .attr(\"height\", tile_height);\n                var inner_svg = sub_svg.append(\"svg\")\n                    .attr(\"x\", 0.0)\n                    .attr(\"y\", 20)\n                    .attr(\"width\", tile_width - 0.0 - 0.0)\n                    .attr(\"height\", tile_height - 20 - 0.0);\n                var frame = inner_svg.append(\"svg\")\n\t\t\t        .attr(\"id\", \"id_8cc2dc6ac6144e56b8bc403ff57ef106_frame\" + i)\n                    //.attr(\"preserveAspectRatio\", \"none\")\n                    .attr(\"viewBox\", \"0.0 0.0 100 100\");\n                i += 1;\n            }\n        }\n    ")

In [9]:
#here(width=200, height=200, trace_width=100., trace_height=100., margin_top=20.)

In [10]:
renderer = JupyterInlineRenderer("agent_model_renderer", Dict("mode" => "overwrite", "show_path" => true))
trace = Trace()
@generate(trace, agent_model())
attach(renderer, figure)
render(renderer, trace) # TODO add legend

In [11]:
set_title!(figure, 1, "A sample")

LoadError: UndefVarError: set_title! not defined

The program is stochastic. We get a sense of the distribution by rendering many frames in an animation:

In [12]:
figure = Figure(width=200, height=200, trace_width=100, trace_height=100, margin_top=20)
here(figure)

HTML{String}("    <div id=\"id_eb7257369a3549f8aa1c3dab1d4031cd\"></div>\n    <script>\n        var d3 = require(\"nbextensions/d3/d3.min\");\n    \n        var svg = d3.select(\"#id_eb7257369a3549f8aa1c3dab1d4031cd\").append(\"svg\")\n            .attr(\"width\", 200)\n            .attr(\"height\", 200);\n    \n        // TODO delete me\n        svg.append(\"rect\")\n            .attr(\"x\", 0)\n            .attr(\"y\", 0)\n            .attr(\"width\", \"100%\")\n            .attr(\"height\", \"100%\")\n            .style(\"stroke\", \"white\")\n            .style(\"fill\", \"white\");\n    \n        var tile_width = 200 / 1;\n        var tile_height = 200 / 1;\n        var i = 1;\n        for (var row=0; row<1; row++) {\n            for (var col=0; col<1; col++) {\n                var sub_svg = svg.append(\"svg\")\n                    .classed(\"subfigure\", true)\n\t\t\t        .attr(\"id\", \"id_eb7257369a3549f8aa1c3dab1d4031cd_parent\" + i)\n                    .attr(\"x\", col * tile_width)\n                    .attr(\"y\", row * tile_height)\n                    .attr(\"width\", tile_width)\n                    .attr(\"height\", tile_height);\n                var inner_svg = sub_svg.append(\"svg\")\n                    .attr(\"x\", 0.0)\n                    .attr(\"y\", 20)\n                    .attr(\"width\", tile_width - 0.0 - 0.0)\n                    .attr(\"height\", tile_height - 20 - 0.0);\n                var frame = inner_svg.append(\"svg\")\n\t\t\t        .attr(\"id\", \"id_eb7257369a3549f8aa1c3dab1d4031cd_frame\" + i)\n                    //.attr(\"preserveAspectRatio\", \"none\")\n                    .attr(\"viewBox\", \"0.0 0.0 100 100\");\n                i += 1;\n            }\n        }\n    ")

In [13]:
attach(renderer, figure)
for i=1:100
    trace = Trace()
    @generate(trace, agent_model())
    render(renderer, trace) # TODO add legend
end

We can also visualize many runs side by side in a grid:

In [14]:
figure = Figure(num_rows=2, num_cols=6, width=900, height=300, trace_width=100, trace_height=100)
here(figure)

HTML{String}("    <div id=\"id_386246cedde0454db023abff205c221d\"></div>\n    <script>\n        var d3 = require(\"nbextensions/d3/d3.min\");\n    \n        var svg = d3.select(\"#id_386246cedde0454db023abff205c221d\").append(\"svg\")\n            .attr(\"width\", 900)\n            .attr(\"height\", 300);\n    \n        // TODO delete me\n        svg.append(\"rect\")\n            .attr(\"x\", 0)\n            .attr(\"y\", 0)\n            .attr(\"width\", \"100%\")\n            .attr(\"height\", \"100%\")\n            .style(\"stroke\", \"white\")\n            .style(\"fill\", \"white\");\n    \n        var tile_width = 900 / 6;\n        var tile_height = 300 / 2;\n        var i = 1;\n        for (var row=0; row<2; row++) {\n            for (var col=0; col<6; col++) {\n                var sub_svg = svg.append(\"svg\")\n                    .classed(\"subfigure\", true)\n\t\t\t        .attr(\"id\", \"id_386246cedde0454db023abff205c221d_parent\" + i)\n                    .attr(\"x\", col * tile_width)\n                    .attr(\"y\", row * tile_height)\n                    .attr(\"width\", tile_width)\n                    .attr(\"height\", tile_height);\n                var inner_svg = sub_svg.append(\"svg\")\n                    .attr(\"x\", 0.0)\n                    .attr(\"y\", 0.0)\n                    .attr(\"width\", tile_width - 0.0 - 0.0)\n                    .attr(\"height\", tile_height - 0.0 - 0.0);\n                var frame = inner_svg.append(\"svg\")\n\t\t\t        .attr(\"id\", \"id_386246cedde0454db023abff205c221d_frame\" + i)\n                    //.attr(\"preserveAspectRatio\", \"none\")\n                    .attr(\"viewBox\", \"0.0 0.0 100 100\");\n                i += 1;\n            }\n        }\n    ")

In [15]:
for i=1:12
    trace = Trace()
    @generate(trace, agent_model())
    attach(renderer, (figure => i))
    render(renderer, trace)
end

We can see what the traces look like when we fix the value of `start` and `destination` with `intervene!`:

In [16]:
figure = Figure(num_rows=2, num_cols=6, width=900, height=300, trace_width=100, trace_height=100)
here(figure)

HTML{String}("    <div id=\"id_5a2b32663bb04c57bf347291d36e006d\"></div>\n    <script>\n        var d3 = require(\"nbextensions/d3/d3.min\");\n    \n        var svg = d3.select(\"#id_5a2b32663bb04c57bf347291d36e006d\").append(\"svg\")\n            .attr(\"width\", 900)\n            .attr(\"height\", 300);\n    \n        // TODO delete me\n        svg.append(\"rect\")\n            .attr(\"x\", 0)\n            .attr(\"y\", 0)\n            .attr(\"width\", \"100%\")\n            .attr(\"height\", \"100%\")\n            .style(\"stroke\", \"white\")\n            .style(\"fill\", \"white\");\n    \n        var tile_width = 900 / 6;\n        var tile_height = 300 / 2;\n        var i = 1;\n        for (var row=0; row<2; row++) {\n            for (var col=0; col<6; col++) {\n                var sub_svg = svg.append(\"svg\")\n                    .classed(\"subfigure\", true)\n\t\t\t        .attr(\"id\", \"id_5a2b32663bb04c57bf347291d36e006d_parent\" + i)\n                    .attr(\"x\", col * tile_width)\n                    .attr(\"y\", row * tile_height)\n                    .attr(\"width\", tile_width)\n                    .attr(\"height\", tile_height);\n                var inner_svg = sub_svg.append(\"svg\")\n                    .attr(\"x\", 0.0)\n                    .attr(\"y\", 0.0)\n                    .attr(\"width\", tile_width - 0.0 - 0.0)\n                    .attr(\"height\", tile_height - 0.0 - 0.0);\n                var frame = inner_svg.append(\"svg\")\n\t\t\t        .attr(\"id\", \"id_5a2b32663bb04c57bf347291d36e006d_frame\" + i)\n                    //.attr(\"preserveAspectRatio\", \"none\")\n                    .attr(\"viewBox\", \"0.0 0.0 100 100\");\n                i += 1;\n            }\n        }\n    ")

In [17]:
for i=1:12
    trace = Trace()
    intervene!(trace, "start", Point(10, 10))
    intervene!(trace, "destination", Point(90, 90))
    @generate(trace, agent_model())
    attach(renderer, (figure => i))
    render(renderer, trace)
end

We can also overlay many traces:

In [18]:
figure = Figure(width=200, height=200, trace_width=100, trace_height=100)
here(figure)

HTML{String}("    <div id=\"id_c92a0b079a5541738b3b4a958f778b42\"></div>\n    <script>\n        var d3 = require(\"nbextensions/d3/d3.min\");\n    \n        var svg = d3.select(\"#id_c92a0b079a5541738b3b4a958f778b42\").append(\"svg\")\n            .attr(\"width\", 200)\n            .attr(\"height\", 200);\n    \n        // TODO delete me\n        svg.append(\"rect\")\n            .attr(\"x\", 0)\n            .attr(\"y\", 0)\n            .attr(\"width\", \"100%\")\n            .attr(\"height\", \"100%\")\n            .style(\"stroke\", \"white\")\n            .style(\"fill\", \"white\");\n    \n        var tile_width = 200 / 1;\n        var tile_height = 200 / 1;\n        var i = 1;\n        for (var row=0; row<1; row++) {\n            for (var col=0; col<1; col++) {\n                var sub_svg = svg.append(\"svg\")\n                    .classed(\"subfigure\", true)\n\t\t\t        .attr(\"id\", \"id_c92a0b079a5541738b3b4a958f778b42_parent\" + i)\n                    .attr(\"x\", col * tile_width)\n                    .attr(\"y\", row * tile_height)\n                    .attr(\"width\", tile_width)\n                    .attr(\"height\", tile_height);\n                var inner_svg = sub_svg.append(\"svg\")\n                    .attr(\"x\", 0.0)\n                    .attr(\"y\", 0.0)\n                    .attr(\"width\", tile_width - 0.0 - 0.0)\n                    .attr(\"height\", tile_height - 0.0 - 0.0);\n                var frame = inner_svg.append(\"svg\")\n\t\t\t        .attr(\"id\", \"id_c92a0b079a5541738b3b4a958f778b42_frame\" + i)\n                    //.attr(\"preserveAspectRatio\", \"none\")\n                    .attr(\"viewBox\", \"0.0 0.0 100 100\");\n                i += 1;\n            }\n        }\n    ")

In [19]:
renderer = JupyterInlineRenderer("agent_model_renderer", Dict("mode" => "overlay", "show_path" => true))
attach(renderer, figure)
for i=1:20
    trace = Trace()
    intervene!(trace, "start", Point(10, 10))
    intervene!(trace, "destination", Point(90, 90))
    @generate(trace, agent_model())
    render(renderer, trace)
end

# 3. Probabilistic inference

So far, we have simulated forward from the model, and we have intervened on some random choices and simulated the consequence. However, suppose we had observed a given sequence of locations of the agent, and we wanted to know probable goal locations? This is a query that cannot be answered simply by forward simulation of the program, because the location of the drone is a *consequence* and not a *cause* of the destination. We can easily find probable consequences given the causes, but finding probable causes given the consequences requires a bit more work.

Here is an example dataset showing measured locations for the first 15 time points:

In [20]:
points = [Point(10, 10), Point(10, 20), Point(10, 30), Point(10, 40)]

4-element Array{Point,1}:
 Point(10.0,10.0)
 Point(10.0,20.0)
 Point(10.0,30.0)
 Point(10.0,40.0)

The first step in inference is to constrain the random choices that are observed.

In [21]:
trace = Trace()
intervene!(trace, "start", Point(10, 10))
for (i, point) in enumerate(points)
    constrain!(trace, "x$i", point.x)
    constrain!(trace, "y$i", point.y)
end

Now, when we run the program in this trace, we find that the score of the trace tells us how well the trace matches the constraints.

In [22]:
figure = Figure(num_rows=3, num_cols=6, width=900, height=450, trace_width=100, trace_height=100)
here(figure)

HTML{String}("    <div id=\"id_801a870cee3a4aed99d27c505a23328d\"></div>\n    <script>\n        var d3 = require(\"nbextensions/d3/d3.min\");\n    \n        var svg = d3.select(\"#id_801a870cee3a4aed99d27c505a23328d\").append(\"svg\")\n            .attr(\"width\", 900)\n            .attr(\"height\", 450);\n    \n        // TODO delete me\n        svg.append(\"rect\")\n            .attr(\"x\", 0)\n            .attr(\"y\", 0)\n            .attr(\"width\", \"100%\")\n            .attr(\"height\", \"100%\")\n            .style(\"stroke\", \"white\")\n            .style(\"fill\", \"white\");\n    \n        var tile_width = 900 / 6;\n        var tile_height = 450 / 3;\n        var i = 1;\n        for (var row=0; row<3; row++) {\n            for (var col=0; col<6; col++) {\n                var sub_svg = svg.append(\"svg\")\n                    .classed(\"subfigure\", true)\n\t\t\t        .attr(\"id\", \"id_801a870cee3a4aed99d27c505a23328d_parent\" + i)\n                    .attr(\"x\", col * tile_width)\n                    .attr(\"y\", row * tile_height)\n                    .attr(\"width\", tile_width)\n                    .attr(\"height\", tile_height);\n                var inner_svg = sub_svg.append(\"svg\")\n                    .attr(\"x\", 0.0)\n                    .attr(\"y\", 0.0)\n                    .attr(\"width\", tile_width - 0.0 - 0.0)\n                    .attr(\"height\", tile_height - 0.0 - 0.0);\n                var frame = inner_svg.append(\"svg\")\n\t\t\t        .attr(\"id\", \"id_801a870cee3a4aed99d27c505a23328d_frame\" + i)\n                    //.attr(\"preserveAspectRatio\", \"none\")\n                    .attr(\"viewBox\", \"0.0 0.0 100 100\");\n                i += 1;\n            }\n        }\n    ")

In [23]:
renderer = JupyterInlineRenderer("agent_model_renderer", Dict("mode" => "overlay",
                                                              "show_path" => true,
                                                              "show_score" => true))
for i=1:30
    t = deepcopy(trace)
    @generate(t, agent_model())
    attach(renderer, (figure => i))
    render(renderer, t)
end

### Importance sampling

We can use this score to filter out traces that don't match well with the observations. Specifically, we can sample a large number of traces, and pick one in proportion to the exponential of its score. We can then repreat this whole process a number of times.

In [24]:
function agent_model_importance_sampling(trace::Trace, num_samples::Int)
    # the input trace contains constraints for the observed data
    traces = Vector{Trace}(num_samples)
    scores = Vector{Float64}(num_samples)
    for k=1:num_samples
        t = deepcopy(trace)
        @generate(t, agent_model())
        scores[k] = score(t)
        traces[k] = t
    end
    weights = exp(scores - logsumexp(scores))
    weights = weights / sum(weights)
    chosen = rand(Categorical(weights))
    return traces[chosen]
end

agent_model_importance_sampling (generic function with 1 method)

In [59]:
num_samples_list = [1, 2, 4, 8, 16]
figure = Figure(num_rows=1, num_cols=5, width=900, height=200, trace_width=100, trace_height=100, margin_top=20,
                titles=map((n) -> "SIR ($n samples)", num_samples_list))
here(figure)

HTML{String}("    <div id=\"id_67e3dbbbb8a642f6953692154f050b85\"></div>\n    <script>\n        var d3 = require(\"nbextensions/d3/d3.min\");\n    \n        var svg = d3.select(\"#id_67e3dbbbb8a642f6953692154f050b85\").append(\"svg\")\n            .attr(\"width\", 900)\n            .attr(\"height\", 200);\n    \n        // TODO delete me\n        svg.append(\"rect\")\n            .attr(\"x\", 0)\n            .attr(\"y\", 0)\n            .attr(\"width\", \"100%\")\n            .attr(\"height\", \"100%\")\n            .style(\"stroke\", \"white\")\n            .style(\"fill\", \"white\");\n    \n        var tile_width = 900 / 5;\n        var tile_height = 200 / 1;\n        var i = 1;\n        for (var row=0; row<1; row++) {\n            for (var col=0; col<5; col++) {\n                var sub_svg = svg.append(\"svg\")\n                    .classed(\"subfigure\", true)\n\t\t\t        .attr(\"id\", \"id_67e3dbbbb8a642f6953692154f050b85_parent\" + i)\n                    .attr(\"x\", col * tile_width)\n                    .attr(\"y\", row * tile_height)\n                    .attr(\"width\", tile_width)\n                    .attr(\"height\", tile_height);\n                var inner_svg = sub_svg.append(\"svg\")\n                    .attr(\"x\", 0.0)\n                    .attr(\"y\", 20)\n                    .attr(\"width\", tile_width - 0.0 - 0.0)\n                    .attr(\"height\", tile_height - 20 - 0.0);\n                var frame = inner_svg.append(\"svg\")\n\t\t\t        .attr(\"id\", \"id_67e3dbbbb8a642f6953692154f050b85_frame\" + i)\n                    //.attr(\"preserveAspectRatio\", \"none\")\n                    .attr(\"viewBox\", \"0.0 0.0 100 100\");\n                i += 1;\n            }\n        }\n    \nvar parent = d3.select(\"#id_67e3dbbbb8a642f6953692154f050b85_parent\" + 1)\nparent.append(\"text\")\n            .attr(\"x\", tile_width / 2)\n            .attr(\"y\", 16)\n            .attr(\"text-anchor\", \"middle\")\n            .text(\"SIR (1 samples)\")\n            .style(\"font-size\", 16);\nvar parent = d3.select(\"#id_67e3dbbbb8a642f6953692154f050b85_parent\" + 2)\nparent.append(\"text\")\n            .attr(\"x\", tile_width / 2)\n            .attr(\"y\", 16)\n            .attr(\"text-anchor\", \"middle\")\n            .text(\"SIR (2 samples)\")\n            .style(\"font-size\", 16);\nvar parent = d3.select(\"#id_67e3dbbbb8a642f6953692154f050b85_parent\" + 3)\nparent.append(\"text\")\n            .attr(\"x\", tile_width / 2)\n            .attr(\"y\", 16)\n            .attr(\"text-anchor\", \"middle\")\n            .text(\"SIR (4 samples)\")\n            .style(\"font-size\", 16);\nvar parent = d3.select(\"#id_67e3dbbbb8a642f6953692154f050b85_parent\" + 4)\nparent.append(\"text\")\n            .attr(\"x\", tile_width / 2)\n            .attr(\"y\", 16)\n            .attr(\"text-anchor\", \"middle\")\n            .text(\"SIR (8 samples)\")\n            .style(\"font-size\", 16);\nvar parent = d3.select(\"#id_67e3dbbbb8a642f6953692154f050b85_parent\" + 5)\nparent.append(\"text\")\n            .attr(\"x\", tile_width / 2)\n            .attr(\"y\", 16)\n            .attr(\"text-anchor\", \"middle\")\n            .text(\"SIR (16 samples)\")\n            .style(\"font-size\", 16);")

In [60]:
renderer = JupyterInlineRenderer("agent_model_renderer", Dict("mode" => "overlay", "show_path" => false,
                                                              "show_score" => false))
num_approximate_samples = 30
trace = Trace()
intervene!(trace, "start", Point(10, 10))
for (i, point) in enumerate(points)
    constrain!(trace, "x$i", point.x)
    constrain!(trace, "y$i", point.y)
end
for (i, num_samples) in enumerate(num_samples_list)
    attach(renderer, (figure => i))
    title =  "SIR ($num_samples particles)"
    println(title)
    for j=1:num_approximate_samples
        output_trace = agent_model_importance_sampling(trace, num_samples)
        render(renderer, output_trace)
    end
end

SIR (1 particles)
SIR (2 particles)
SIR (4 particles)
SIR (8 particles)
SIR (16 particles)


### Metropolis-Hastings Inference

In [69]:
renderer = JupyterInlineRenderer("agent_model_renderer", Dict("mode" => "overwrite",
                                                              "show_path" => true,
                                                              "show_score" => true))
figure = Figure(num_rows=1, num_cols=2, width=400, height=200, trace_width=100, trace_height=100,
                margin_top=20, titles=["proposed trace", "current trace"])
here(figure)

HTML{String}("    <div id=\"id_43b1a93b79fe48eca43fe6ea76655e3f\"></div>\n    <script>\n        var d3 = require(\"nbextensions/d3/d3.min\");\n    \n        var svg = d3.select(\"#id_43b1a93b79fe48eca43fe6ea76655e3f\").append(\"svg\")\n            .attr(\"width\", 400)\n            .attr(\"height\", 200);\n    \n        // TODO delete me\n        svg.append(\"rect\")\n            .attr(\"x\", 0)\n            .attr(\"y\", 0)\n            .attr(\"width\", \"100%\")\n            .attr(\"height\", \"100%\")\n            .style(\"stroke\", \"white\")\n            .style(\"fill\", \"white\");\n    \n        var tile_width = 400 / 2;\n        var tile_height = 200 / 1;\n        var i = 1;\n        for (var row=0; row<1; row++) {\n            for (var col=0; col<2; col++) {\n                var sub_svg = svg.append(\"svg\")\n                    .classed(\"subfigure\", true)\n\t\t\t        .attr(\"id\", \"id_43b1a93b79fe48eca43fe6ea76655e3f_parent\" + i)\n                    .attr(\"x\", col * tile_width)\n                    .attr(\"y\", row * tile_height)\n                    .attr(\"width\", tile_width)\n                    .attr(\"height\", tile_height);\n                var inner_svg = sub_svg.append(\"svg\")\n                    .attr(\"x\", 0.0)\n                    .attr(\"y\", 20)\n                    .attr(\"width\", tile_width - 0.0 - 0.0)\n                    .attr(\"height\", tile_height - 20 - 0.0);\n                var frame = inner_svg.append(\"svg\")\n\t\t\t        .attr(\"id\", \"id_43b1a93b79fe48eca43fe6ea76655e3f_frame\" + i)\n                    //.attr(\"preserveAspectRatio\", \"none\")\n                    .attr(\"viewBox\", \"0.0 0.0 100 100\");\n                i += 1;\n            }\n        }\n    \nvar parent = d3.select(\"#id_43b1a93b79fe48eca43fe6ea76655e3f_parent\" + 1)\nparent.append(\"text\")\n            .attr(\"x\", tile_width / 2)\n            .attr(\"y\", 16)\n            .attr(\"text-anchor\", \"middle\")\n            .text(\"proposed trace\")\n            .style(\"font-size\", 16);\nvar parent = d3.select(\"#id_43b1a93b79fe48eca43fe6ea76655e3f_parent\" + 2)\nparent.append(\"text\")\n            .attr(\"x\", tile_width / 2)\n            .attr(\"y\", 16)\n            .attr(\"text-anchor\", \"middle\")\n            .text(\"current trace\")\n            .style(\"font-size\", 16);")

In [70]:
current_trace = Trace()
intervene!(current_trace, "start", Point(10, 10))
for (i, point) in enumerate(points)
    constrain!(current_trace, "x$i", point.x)
    constrain!(current_trace, "y$i", point.y)
end
@generate(current_trace, agent_model())
current_score = score(current_trace)
for i=1:100
    proposed_trace = deepcopy(current_trace)
    @generate(proposed_trace, agent_model())
    proposed_score = score(proposed_trace)
    if log(rand()) < proposed_score - current_score
        current_trace = proposed_trace
        current_score = proposed_score
    end
    attach(renderer, figure => 1)
    render(renderer, proposed_trace)
    attach(renderer, figure => 2)
    render(renderer, current_trace)
   # sleep(0.1)
end

# 4. Improving the model

Our model above made a lot of assumptions that are unlikely to hold in the real world. For example, the agent always takes pretty direct paths from its starting location to its final destination. What if the agent is more unpredictable? What if it takes detours?

Here is a dataset that does not match our model's expectations.

In [33]:
xs = [9.59825,21.8936,30.9534,43.1137,48.8929,46.0282,35.0281,27.2084,20.1662,18.7309]
ys = [8.92063,9.54817,10.8819,9.75395,10.4189,21.7662,25.9994,33.5729,39.9398,50.0026];

In [31]:
figure = Figure(width=200, height=200, trace_width=100, trace_height=100)
here(figure)

HTML{String}("    <div id=\"id_6c26da9c00934e09b7ceba53fec44a18\"></div>\n    <script>\n        var d3 = require(\"nbextensions/d3/d3.min\");\n    \n        var svg = d3.select(\"#id_6c26da9c00934e09b7ceba53fec44a18\").append(\"svg\")\n            .attr(\"width\", 200)\n            .attr(\"height\", 200);\n    \n        // TODO delete me\n        svg.append(\"rect\")\n            .attr(\"x\", 0)\n            .attr(\"y\", 0)\n            .attr(\"width\", \"100%\")\n            .attr(\"height\", \"100%\")\n            .style(\"stroke\", \"white\")\n            .style(\"fill\", \"white\");\n    \n        var tile_width = 200 / 1;\n        var tile_height = 200 / 1;\n        var i = 1;\n        for (var row=0; row<1; row++) {\n            for (var col=0; col<1; col++) {\n                var sub_svg = svg.append(\"svg\")\n                    .classed(\"subfigure\", true)\n\t\t\t        .attr(\"id\", \"id_6c26da9c00934e09b7ceba53fec44a18_parent\" + i)\n                    .attr(\"x\", col * tile_width)\n                    .attr(\"y\", row * tile_height)\n                    .attr(\"width\", tile_width)\n                    .attr(\"height\", tile_height);\n                var inner_svg = sub_svg.append(\"svg\")\n                    .attr(\"x\", 0.0)\n                    .attr(\"y\", 0.0)\n                    .attr(\"width\", tile_width - 0.0 - 0.0)\n                    .attr(\"height\", tile_height - 0.0 - 0.0);\n                var frame = inner_svg.append(\"svg\")\n\t\t\t        .attr(\"id\", \"id_6c26da9c00934e09b7ceba53fec44a18_frame\" + i)\n                    //.attr(\"preserveAspectRatio\", \"none\")\n                    .attr(\"viewBox\", \"0.0 0.0 100 100\");\n                i += 1;\n            }\n        }\n    ")

In [35]:
renderer = JupyterInlineRenderer("agent_model_renderer", Dict("mode" => "overwrite",
                                                              "show_path" => true,
                                                              "show_score" => false))
attach(renderer, figure)
trace = Trace()
intervene!(trace, "start", Point(10, 10))
for (i, (x, y)) in enumerate(zip(xs, ys))
    constrain!(trace, "x$i", x)
    constrain!(trace, "y$i", y)
end
@generate(trace, agent_model())
delete!(trace, "start")
delete!(trace, "destination")
# TODO: only show the consrained data (not the other points on a randomly sampled path)
render(renderer, trace)

 Let's see what happens when we try to do probabilistic inference in our model, given this data.

In [67]:
all_num_simulations = [1, 2, 4, 8, 16]#, 32, 64, 128]
figure = Figure(num_rows=1, num_cols=5, width=900, height=200, trace_width=100, trace_height=100,
                margin_top=20, titles=map((n) -> "SIR ($n particles)", all_num_simulations))
here(figure)

HTML{String}("    <div id=\"id_86bead1eb812445e8d3aab58d77c9826\"></div>\n    <script>\n        var d3 = require(\"nbextensions/d3/d3.min\");\n    \n        var svg = d3.select(\"#id_86bead1eb812445e8d3aab58d77c9826\").append(\"svg\")\n            .attr(\"width\", 900)\n            .attr(\"height\", 200);\n    \n        // TODO delete me\n        svg.append(\"rect\")\n            .attr(\"x\", 0)\n            .attr(\"y\", 0)\n            .attr(\"width\", \"100%\")\n            .attr(\"height\", \"100%\")\n            .style(\"stroke\", \"white\")\n            .style(\"fill\", \"white\");\n    \n        var tile_width = 900 / 5;\n        var tile_height = 200 / 1;\n        var i = 1;\n        for (var row=0; row<1; row++) {\n            for (var col=0; col<5; col++) {\n                var sub_svg = svg.append(\"svg\")\n                    .classed(\"subfigure\", true)\n\t\t\t        .attr(\"id\", \"id_86bead1eb812445e8d3aab58d77c9826_parent\" + i)\n                    .attr(\"x\", col * tile_width)\n                    .attr(\"y\", row * tile_height)\n                    .attr(\"width\", tile_width)\n                    .attr(\"height\", tile_height);\n                var inner_svg = sub_svg.append(\"svg\")\n                    .attr(\"x\", 0.0)\n                    .attr(\"y\", 20)\n                    .attr(\"width\", tile_width - 0.0 - 0.0)\n                    .attr(\"height\", tile_height - 20 - 0.0);\n                var frame = inner_svg.append(\"svg\")\n\t\t\t        .attr(\"id\", \"id_86bead1eb812445e8d3aab58d77c9826_frame\" + i)\n                    //.attr(\"preserveAspectRatio\", \"none\")\n                    .attr(\"viewBox\", \"0.0 0.0 100 100\");\n                i += 1;\n            }\n        }\n    \nvar parent = d3.select(\"#id_86bead1eb812445e8d3aab58d77c9826_parent\" + 1)\nparent.append(\"text\")\n            .attr(\"x\", tile_width / 2)\n            .attr(\"y\", 16)\n            .attr(\"text-anchor\", \"middle\")\n            .text(\"SIR (1 particles)\")\n            .style(\"font-size\", 16);\nvar parent = d3.select(\"#id_86bead1eb812445e8d3aab58d77c9826_parent\" + 2)\nparent.append(\"text\")\n            .attr(\"x\", tile_width / 2)\n            .attr(\"y\", 16)\n            .attr(\"text-anchor\", \"middle\")\n            .text(\"SIR (2 particles)\")\n            .style(\"font-size\", 16);\nvar parent = d3.select(\"#id_86bead1eb812445e8d3aab58d77c9826_parent\" + 3)\nparent.append(\"text\")\n            .attr(\"x\", tile_width / 2)\n            .attr(\"y\", 16)\n            .attr(\"text-anchor\", \"middle\")\n            .text(\"SIR (4 particles)\")\n            .style(\"font-size\", 16);\nvar parent = d3.select(\"#id_86bead1eb812445e8d3aab58d77c9826_parent\" + 4)\nparent.append(\"text\")\n            .attr(\"x\", tile_width / 2)\n            .attr(\"y\", 16)\n            .attr(\"text-anchor\", \"middle\")\n            .text(\"SIR (8 particles)\")\n            .style(\"font-size\", 16);\nvar parent = d3.select(\"#id_86bead1eb812445e8d3aab58d77c9826_parent\" + 5)\nparent.append(\"text\")\n            .attr(\"x\", tile_width / 2)\n            .attr(\"y\", 16)\n            .attr(\"text-anchor\", \"middle\")\n            .text(\"SIR (16 particles)\")\n            .style(\"font-size\", 16);")

In [68]:
trace = Trace()
intervene!(trace, "start", Point(10, 10))
for (i, (x, y)) in enumerate(zip(xs, ys))
    constrain!(trace, "x$i", x)
    constrain!(trace, "y$i", y)
end
renderer = JupyterInlineRenderer("agent_model_renderer", Dict("mode" => "overlay",
                                                              "show_path" => true,
                                                              "show_score" => false))
num_approximate_samples = 50
for (i, num_simulations) in enumerate(all_num_simulations)
    attach(renderer, figure => i)
    for j=1:num_approximate_samples
        traces = Vector{Trace}(num_simulations)
        scores = Vector{Float64}(num_simulations)
        for k=1:num_simulations
            t = deepcopy(trace)
            @generate(t, agent_model())
            scores[k] = score(t)
            traces[k] = t
        end
        weights = exp(scores - logsumexp(scores))
        weights = weights / sum(weights)
        chosen = rand(Categorical(weights))
        render(renderer, traces[chosen])
    end
end

The inferences do not look intuitive. This is because the model `agent_model` cannot explain the detour. It assumes that the detour must be the goal, an it explains the observed data as a very unlikely accident of noise. This is an example of model mis-specification.  In order to make reasonablee inferences for datasets that may contain a detour, we use an improved model program, shown below:

In [41]:
@program agent_waypoint_model() begin
    
    # assumed scene
    scene = Scene(0, 100, 0, 100) # the scene spans the square [0, 100] x [0, 100]
    add!(scene, Tree(Point(30, 20))) # place a tree at x=30, y=20
    add!(scene, Tree(Point(83, 80)))
    add!(scene, Tree(Point(80, 40)))
    wall_height = 30.
    add!(scene, Wall(Point(20., 40.), 1, 40., 2., wall_height))
    add!(scene, Wall(Point(60., 40.), 2, 40., 2., wall_height))
    add!(scene, Wall(Point(60.-15., 80.), 1, 15. + 2., 2., wall_height))
    add!(scene, Wall(Point(20., 80.), 1, 15., 2., wall_height))
    add!(scene, Wall(Point(20., 40.), 2, 40., 2., wall_height))    
    
    # time points at which we observe the agent's location
    observation_times = collect(linspace(0.0, 200.0, 20)) ~ "times"
    
    # assumed speed of the agent
    speed = 1.0
    
    # the starting location of the agent is a random point in the scene
    start = Point(uniform(0, 100), uniform(0, 100)) ~ "start"
    
    # the destination of the agent is a random point in the scene
    destination = Point(uniform(0, 100), uniform(0, 100)) ~ "destination"
    
    if (flip(0.5) ~ "use-waypoint")
        waypoint = Point(uniform(0, 100) ~ "waypoint-x", uniform(0, 100) ~ "waypoint-y")
        (tree1, rough_path1, final_path1) = plan_path(start, waypoint, scene)
        (tree2, rough_path2, final_path2) = plan_path(waypoint, destination, scene)
        
        # if either path planner sub-problem failed, then no path was found (final_path is null)
        if isnull(final_path1) || isnull(final_path2)
            final_path = Nullable{Path}() # null
        else
            final_path = Nullable{Path}(concatenate(get(final_path1), get(final_path2)))
        end
    else
        (tree, rough_path, final_path) = plan_path(start, destination, scene)
    end
    
    # the path of the agent from its start location to its destination
    # uses a simple 2D holonomic path planner based on RRT (path_planner.jl)
    
    if isnull(final_path)
        
        # the agent could not find a path to its destination
        # assume it stays at the start location indefinitely
        locations = [start for _ in observation_times]
    else
        
        # the agent found a path to its destination
        # assume it moves from the start to the destinatoin along the path at constnat speed
        # sample its location along this path for each time in observation times
        locations = walk_path(get(final_path), speed, observation_times)
    end
    
    # assume that the observed locations are noisy measurements of the true locations
    # assume the noise is normally distributed with standard deviation 'noise'
    noise = 1.0
    for (i, t) in enumerate(observation_times)
        measured_x = normal(locations[i].x, noise) ~ "x$i"
        measured_y = normal(locations[i].y, noise) ~ "y$i"
    end
    
    # record other program state for rendering
    scene ~ "scene"
end;

Here are some simulations from the program, for a fixed start and destination.

In [43]:
figure = Figure(num_rows=3, num_cols=6, width=900, height=450, trace_width=100, trace_height=100)
here(figure)

HTML{String}("    <div id=\"id_3bfcc6e6b15043b8b6b11f9215a89301\"></div>\n    <script>\n        var d3 = require(\"nbextensions/d3/d3.min\");\n    \n        var svg = d3.select(\"#id_3bfcc6e6b15043b8b6b11f9215a89301\").append(\"svg\")\n            .attr(\"width\", 900)\n            .attr(\"height\", 450);\n    \n        // TODO delete me\n        svg.append(\"rect\")\n            .attr(\"x\", 0)\n            .attr(\"y\", 0)\n            .attr(\"width\", \"100%\")\n            .attr(\"height\", \"100%\")\n            .style(\"stroke\", \"white\")\n            .style(\"fill\", \"white\");\n    \n        var tile_width = 900 / 6;\n        var tile_height = 450 / 3;\n        var i = 1;\n        for (var row=0; row<3; row++) {\n            for (var col=0; col<6; col++) {\n                var sub_svg = svg.append(\"svg\")\n                    .classed(\"subfigure\", true)\n\t\t\t        .attr(\"id\", \"id_3bfcc6e6b15043b8b6b11f9215a89301_parent\" + i)\n                    .attr(\"x\", col * tile_width)\n                    .attr(\"y\", row * tile_height)\n                    .attr(\"width\", tile_width)\n                    .attr(\"height\", tile_height);\n                var inner_svg = sub_svg.append(\"svg\")\n                    .attr(\"x\", 0.0)\n                    .attr(\"y\", 0.0)\n                    .attr(\"width\", tile_width - 0.0 - 0.0)\n                    .attr(\"height\", tile_height - 0.0 - 0.0);\n                var frame = inner_svg.append(\"svg\")\n\t\t\t        .attr(\"id\", \"id_3bfcc6e6b15043b8b6b11f9215a89301_frame\" + i)\n                    //.attr(\"preserveAspectRatio\", \"none\")\n                    .attr(\"viewBox\", \"0.0 0.0 100 100\");\n                i += 1;\n            }\n        }\n    ")

In [45]:
renderer = JupyterInlineRenderer("agent_model_renderer", Dict("mode" => "overwrite", "show_path" => true))
traces = []
for i=1:18
    trace = Trace()
    intervene!(trace, "start", Point(10, 10))
    intervene!(trace, "destination", Point(90, 90))
    @generate(trace, agent_waypoint_model())
    attach(renderer, figure => i)
    render(renderer, trace)
    push!(traces, trace)
end

Notice that sometimes, the path has a clear waypoint/detour whereas other times it does not.

We now do inference in this model for the given dataset.

In [49]:
all_num_simulations = [1, 4, 16, 64, 256]#, 1024]
figure = Figure(num_rows=1, num_cols=5, width=900, height=200, trace_width=100, trace_height=100,
                margin_top=20, titles=map((n) -> "SIR ($n particles)", all_num_simulations))
here(figure)

HTML{String}("    <div id=\"id_5310cd33978846378c50994496dbb75d\"></div>\n    <script>\n        var d3 = require(\"nbextensions/d3/d3.min\");\n    \n        var svg = d3.select(\"#id_5310cd33978846378c50994496dbb75d\").append(\"svg\")\n            .attr(\"width\", 900)\n            .attr(\"height\", 150);\n    \n        // TODO delete me\n        svg.append(\"rect\")\n            .attr(\"x\", 0)\n            .attr(\"y\", 0)\n            .attr(\"width\", \"100%\")\n            .attr(\"height\", \"100%\")\n            .style(\"stroke\", \"white\")\n            .style(\"fill\", \"white\");\n    \n        var tile_width = 900 / 6;\n        var tile_height = 150 / 1;\n        var i = 1;\n        for (var row=0; row<1; row++) {\n            for (var col=0; col<6; col++) {\n                var sub_svg = svg.append(\"svg\")\n                    .classed(\"subfigure\", true)\n\t\t\t        .attr(\"id\", \"id_5310cd33978846378c50994496dbb75d_parent\" + i)\n                    .attr(\"x\", col * tile_width)\n                    .attr(\"y\", row * tile_height)\n                    .attr(\"width\", tile_width)\n                    .attr(\"height\", tile_height);\n                var inner_svg = sub_svg.append(\"svg\")\n                    .attr(\"x\", 0.0)\n                    .attr(\"y\", 20)\n                    .attr(\"width\", tile_width - 0.0 - 0.0)\n                    .attr(\"height\", tile_height - 20 - 0.0);\n                var frame = inner_svg.append(\"svg\")\n\t\t\t        .attr(\"id\", \"id_5310cd33978846378c50994496dbb75d_frame\" + i)\n                    //.attr(\"preserveAspectRatio\", \"none\")\n                    .attr(\"viewBox\", \"0.0 0.0 100 100\");\n                i += 1;\n            }\n        }\n    \nvar parent = d3.select(\"#id_5310cd33978846378c50994496dbb75d_parent\" + 1)\nparent.append(\"text\")\n            .attr(\"x\", tile_width / 2)\n            .attr(\"y\", 16)\n            .attr(\"text-anchor\", \"middle\")\n            .text(\"SIR (1 particles)\")\n            .style(\"font-size\", 16);\nvar parent = d3.select(\"#id_5310cd33978846378c50994496dbb75d_parent\" + 2)\nparent.append(\"text\")\n            .attr(\"x\", tile_width / 2)\n            .attr(\"y\", 16)\n            .attr(\"text-anchor\", \"middle\")\n            .text(\"SIR (4 particles)\")\n            .style(\"font-size\", 16);\nvar parent = d3.select(\"#id_5310cd33978846378c50994496dbb75d_parent\" + 3)\nparent.append(\"text\")\n            .attr(\"x\", tile_width / 2)\n            .attr(\"y\", 16)\n            .attr(\"text-anchor\", \"middle\")\n            .text(\"SIR (16 particles)\")\n            .style(\"font-size\", 16);\nvar parent = d3.select(\"#id_5310cd33978846378c50994496dbb75d_parent\" + 4)\nparent.append(\"text\")\n            .attr(\"x\", tile_width / 2)\n            .attr(\"y\", 16)\n            .attr(\"text-anchor\", \"middle\")\n            .text(\"SIR (64 particles)\")\n            .style(\"font-size\", 16);\nvar parent = d3.select(\"#id_5310cd33978846378c50994496dbb75d_parent\" + 5)\nparent.append(\"text\")\n            .attr(\"x\", tile_width / 2)\n            .attr(\"y\", 16)\n            .attr(\"text-anchor\", \"middle\")\n            .text(\"SIR (256 particles)\")\n            .style(\"font-size\", 16);")

In [50]:


trace = Trace()
intervene!(trace, "start", Point(10, 10))
for (i, (x, y)) in enumerate(zip(xs, ys))
    constrain!(trace, "x$i", x)
    constrain!(trace, "y$i", y)
end
renderer = JupyterInlineRenderer("agent_model_renderer", Dict("mode" => "overlay",
                                                              "show_path" => true,
                                                              "show_score" => false))
num_approximate_samples = 10
for (i, num_simulations) in enumerate(all_num_simulations)
    attach(renderer, figure => i)
    for j=1:num_approximate_samples
        traces = Vector{Trace}(num_simulations)
        scores = Vector{Float64}(num_simulations)
        for k=1:num_simulations
            t = deepcopy(trace)
            @generate(t, agent_waypoint_model())
            scores[k] = score(t)
            traces[k] = t
        end
        weights = exp(scores - logsumexp(scores))
        weights = weights / sum(weights)
        chosen = rand(Categorical(weights))
        render(renderer, traces[chosen])
    end
end

Notice that it's possible to get reasonalbe inferences, but it takes a lot longer than with the previous model.

It looks like the results are not accurate. This is because in the new model, a random forward execution of the imporved model is a lot less likely to match the observations than a random forward exection of the original program. We can try to increase the number of samples to incrase the probability that we get one that matches the data. Note that this will take a few minutes t orun:

In [41]:
# TODO: Show results with a larger number of particles (256) should look

This modification of the model mad einference a lot more computationally challenging, and our importance sampling algorihtm is not able to give us real-time inferneces. This motivates the need for a more sophisicated approach to probabilisitc inference.

# 5. Compiling inference with neural networks

There are a number of approaches for creating more efficient inference algorithms. We will focus on one approach, where we train a neural network to make informed guesses about he locatio nof the waypoint. First, let's understand in a bit more detail why the default inference algorihtm was slow

Suppose we knew the right waypoint:

In [42]:
#trace = Trace()
#constrain!(trace, "use-waypoint", true)
#constrain!(trace, "waypoint", Point(0.5, 0.5))

Then the baseline importance sapmling algorithm gives reasonable inferences with fewer samples. We use this idea by training a neural network to make informed guesses about the waypoint, given the observed data as its input. We train the neural network on nany simulatoins of the program. Then, the resulting trained neural network can be used to make informed guesses about the waypooint given any observations.

In [43]:
# show the neural network, and show the training.

Let's visualize the guesses made by the neural network for a few  different datasets:

In [44]:
# show four renderings left to right of different datasets, with circles denoting the neural network's guess about the waypoint. 

Now, let's use this trained neural network to speed up inference:

In [45]:
# show the modififed SIR algorithm, using propose!

In [46]:
# show results for fewer particles, which should be noticeably faster than without the neural network.
# make an explicit comparison.